<a target="_blank" href="https://colab.research.google.com/github/ddmms/ml-peg/blob/main/docs/source/tutorials/python/adding_benchmark.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Adding a benchmark to ML-PEG

This notebook guides you through the process of adding a new benchmark to ML-PEG with interactive capabilities. The benchmark added will have the simplest form of interactivity, which is table -> scatter -> structure, meaning clicking a table cell shows a scatter plot from which the errors were calculated, and clicking a point on the scatter plot shows the corresponding structure. Different types (e.g. density plots) or more advanced interactivity (table -> scatter -> phonon dispersion) can also be achieved but will not be covered in this first introduction.

The workflow consists of 3 main steps + documentation:
- **calc**: where the benchmarking script is defined, typically outputting results as .xyz files (e.g. energies for a structure are stored as properties in the .xyz file)
- **analysis**: takes the outputs from calc, computes relevant metrics, builds tables and plots (saved as .json files).
- **app**: builds the interactive app using the analysis output plots/tables and structure files.

---

## 0. Set up

### 0.0 Setting up the environment

If you are following this tutorial using **Google Colab**, uncomment the following cells to clone the repository and install ML-PEG and its dependencies.

In [ ]:
# ! git clone https://github.com/ddmms/ml-peg
# %cd ml-peg

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# ! pip uninstall numpy -y # Uninstall pre-installed numpy

# ! pip uninstall torch torchaudio torchvision transformers -y # Uninstall pre-installed torch
# ! uv pip install torch==2.5.1 # Install pinned version of torch

# ! uv pip install ml-peg[mace,orb,d3] --system # Install ML-PEG with MACE, Orb, and torch-dftd

# get_ipython().kernel.do_shutdown(restart=True) # Restart kernel to update libraries. This may warn that your session has crashed.

If you have cloned the repository and are following this tutorial **locally**, we strongly recommend using `uv sync` as described in our [online documentation](https://ddmms.github.io/ml-peg/developer_guide/get_started.html).

For troubleshooting, please also refer to the `uv` documentation interactions with [Jupyter/VS Code](https://docs.astral.sh/uv/guides/integration/jupyter/).

Installing our pre-commit hooks is recommended, as this helps ensure contributions meet our guidelines. This requires additional dependencies that are installed by `uv sync`, but are not included when running `uv pip install ml_peg` in the cells above.

In [ ]:
! pre-commit install

### 0.1 GitHub

To contribute a new benchmark, first open an issue on GitHub by going to Issues -> New issue -> New benchmark, where you can briefly describe the benchmark you wish to add. After you have created the issue, click on the issue and assign yourself to it (top RHS of page under "Assignees").

You can then create a new branch for your benchmark:

In [ ]:
! git checkout main
! git pull origin main
! git checkout -b my_new_benchmark

### 0.2 Setting up your directories

In the ML-PEG, we organise benchmarks into *categories*, which make up the tabs in the app, along with the summary page. For example, the Molecular Crystals category consists of molecular crystal benchmarks e.g. DMC-ICE13 and X23.

First see if your benchmark fits into an existing category. If so, create a new benchmark folder in the relevant category folders in `ml_peg/calcs/<category>/`, `ml_peg/analysis/<category>/` and `ml_peg/app/<category>/`, and move onto the next steps.

If not, create a new category folder in `ml_peg/calcs/`, `ml_peg/analysis/` and `ml_peg/app/`. You'll also need to create a simple app config file in `ml_peg/app/<category>/` e.g. `ml_peg/app/molecular_crystal/molecular_crystal.yml`:
```yaml
    title: Molecular Crystals
    description: Formation energies of molecular crystals
```
and add the category directory name to `docs/source/user_guide/benchmarks/index.rst` so that it appears in the documentation sidebar.

### 0.3 Documentation
Before adding a new benchmark, it is good practice to add a documentation page for it. You can follow the format of existing benchmark documentation pages in `docs/source/user_guide/benchmarks/`. This is the example for the X23 molecular crystal benchmark we will look at in this notebook `docs/source/user_guide/benchmarks/molecular_crystals.rst`: 

```rst
    ==================
    Molecular Crystals
    ==================

    X23
    ===

    Summary
    -------

    Performance in predicting lattice energies for 23 molecular crystals.


    Metrics
    -------

    1. Lattice energy error

    Accuracy of lattice energy predictions.

    For each molecular crystal, lattice energy is calculated by taking the difference
    between the energy of the solid molecular crystal divided by the number of molecules it
    comprises, and the energy of the isolated molecule. This is compared to the reference
    lattice energy.


    Computational cost
    ------------------

    Low: tests are likely to take less than a minute to run on CPU.


    Data availability
    -----------------

    Input structures:

    * A. M. Reilly and A. Tkatchenko, Understanding the role of vibrations, exact exchange,
    and many-body van der waals interactions in the cohesive properties of molecular
    crystals, The Journal of chemical physics 139 (2013).

    Reference data:

    * Same as input data
    * DMC
```

---

## 1. Calc step: defining the benchmarking script

First create a directory for your benchmark in `ml_peg/calcs/<category>/your_benchmark/` and create a new Python script `calc_your_benchmark.py` in this directory. This is where you will define your benchmarking script. Please use underscores, not hyphens, in the benchmark name to avoid import errors when building the final app.

Now prepare your imports, load models (from `models/models.yml`), define data and output paths, and any unit conversions you may need. This setup is fairly standard across all benchmarks.

In [ ]:
"""Run calculations for X23 tests."""

from __future__ import annotations

from copy import copy
from pathlib import Path
from typing import Any

from ase import units
from ase.io import read, write
import numpy as np
import pytest

from ml_peg.calcs.utils.utils import download_s3_data
from ml_peg.models.get_models import load_models
from ml_peg.models.models import current_models

MODELS = load_models(current_models)

DATA_PATH = Path(__file__).parent / "data"
OUT_PATH = Path(__file__).parent / "outputs"

# Unit conversion
EV_TO_KJ_PER_MOL = units.mol / units.kJ

Now you can define your benchmarking function. We use `pytest` to run all benchmarks in ML-PEG, so the function **must** begin with `test_` to be automatically discovered by `pytest`. A `pytest` decorator is also used to loop this test over all specified models.

For this example, `add_d3_calculator` is used to add dispersion corrections to models where applicable. This is being expanded to include other dispersion corrections.

Input data: We typically store data required to run calculations in an S3 bucket, which can be accessed using the `download_s3_data` function. As this uses the cache to prevent unnecessary downloads, for development you can place a zipped version of your input data in your cache folder e.g. `"~/.cache/ml-peg/lattice_energy.zip"` to test this function before the data has been uploaded.

When the benchmark is finalised and ready to be merged with the main branch, please ask for your data to be uploaded to the S3 bucket, for easy data download by users. This will be saved in the form `inputs/<category>/your_benchmark/your_benchmark.zip`, even if `your_benchmark.zip` differs to the original filename as in this case.

In the test below, after loading the dataset, the benchmark loops over all structures, performing single point calculations. The results are then stored in the `atoms.info`, and the structures are saved as .xyz files. Saving the results in this way allows for easy structure visualisation in the app later.

Importantly, **no analysis is done here**, only the calculation and saving of energies (other examples may save forces, stresses, phonon frequencies etc.). All analysis is done in the analysis step.

In [ ]:
@pytest.mark.parametrize("mlip", MODELS.items())
def test_lattice_energy(mlip: tuple[str, Any]) -> None:
    """
    Run X23 lattice energy test.

    Parameters
    ----------
    mlip
        Name of model use and model to get calculator.
    """
    model_name, model = mlip
    calc = model.get_calculator()

    # Add D3 calculator for this test (for models where applicable)
    calc = model.add_d3_calculator(calc)

    # Download X23 dataset
    lattice_energy_dir = (
        download_s3_data(
            key="inputs/molecular_crystal/X23/X23.zip",
            filename="lattice_energy.zip",
        )
        / "lattice_energy"
    )

    with open(lattice_energy_dir / "list") as f:
        systems = f.read().splitlines()

    for system in systems:
        molecule_path = lattice_energy_dir / system / "POSCAR_molecule"
        solid_path = lattice_energy_dir / system / "POSCAR_solid"
        ref_path = lattice_energy_dir / system / "lattice_energy_DMC"
        num_molecules_path = lattice_energy_dir / system / "nmol"

        molecule = read(molecule_path, index=0, format="vasp")
        molecule.calc = calc
        molecule.get_potential_energy()

        solid = read(solid_path, index=0, format="vasp")
        solid.calc = copy(calc)
        solid.get_potential_energy()

        ref = np.loadtxt(ref_path)[0]
        num_molecules = np.loadtxt(num_molecules_path)

        solid.info["ref"] = ref
        solid.info["num_molecules"] = num_molecules
        solid.info["system"] = system
        molecule.info["ref"] = ref
        molecule.info["num_molecules"] = num_molecules
        molecule.info["system"] = system

        # Write output structures
        write_dir = OUT_PATH / model_name
        write_dir.mkdir(parents=True, exist_ok=True)
        write(write_dir / f"{system}.xyz", [solid, molecule])


You can now run your benchmark through the CLI:

```bash
    ml_peg calc --category molecular_crystal --test X23
```
See the [CLI documentation](https://ddmms.github.io/ml-peg/developer_guide/running.html) for more details.

Once the benchmark is run, you should see your output files in `ml_peg/calcs/<category>/your_benchmark/outputs/`,structured in the following way:
```bash
    calcs/
    └── molecular_crystal/
        └── X23/
            └── outputs/
                └── mace-mp-0a/
                    └── structure_1.xyz
                    └── structure_2.xyz
                    └── ...
                └── mace-mp-0b3/
                    └── structure_1.xyz
                    └── structure_2.xyz
                    └── ...
                └── ...
```

### 1.1 Adding pre-computed data

In some cases, you may wish to add pre-computed data to ML-PEG without defining a calc script. This is not encouraged for benchmarks which fall into the short, medium or long compute time categories, but may be acceptable for very long compute time benchmarks where data generation is very expensive.

In this case, you can create the output directory structure as above, and place your pre-computed files in the relevant model sub-directories.

## 2. Analysis step: computing metrics and building plots/tables

In the analysis step, we take the calc outputs and prepare tables and plots for the app. First create a new directory for your benchmark in `ml_peg/analysis/<category>/your_benchmark/` and create a new python script `analyse_your_benchmark.py` in this directory.

You also need to define `metrics.yml` in the same directory, specifying the metrics you compute, good and bad normalisation thresholds, units, a tooltip (column title hover text) and the level of theory of the reference data for this metric. If the metric is unitless, use `unit: null`. The example for X23 in `analysis/molecular_crystal/X23/metrics.yml`:
```yaml
    metrics:
        MAE:
            good: 0.0
            bad: 100.0
            unit: kJ/mol
            tooltip: "Mean Absolute Error for all systems"
            level_of_theory: DMC
```
For more information on the good and bad thresholds (what are they and how to choose defaults), see the [normalisation documentation](https://ddmms.github.io/ml-peg/developer_guide/scoring_and_normalisation.html).

Again, the imports are fairly standard across benchmarks, along with loading models and defining data/output paths. The key difference is for benchmarks which use dispersion corrections, its required to define `D3_MODEL_NAMES = build_d3_name_map(MODELS)` so that model names appear with `+D3` suffix in the app table.

In [ ]:
"""Analyse X23 benchmark."""

from __future__ import annotations

from pathlib import Path

from ase import units
from ase.io import read, write
import pytest

from ml_peg.analysis.utils.decorators import build_table, plot_parity
from ml_peg.analysis.utils.utils import build_d3_name_map, load_metrics_config, mae
from ml_peg.app import APP_ROOT
from ml_peg.calcs import CALCS_ROOT
from ml_peg.models.get_models import get_model_names
from ml_peg.models.models import current_models

MODELS = get_model_names(current_models)
D3_MODEL_NAMES = build_d3_name_map(MODELS)
CALC_PATH = CALCS_ROOT / "molecular_crystal" / "X23" / "outputs"
OUT_PATH = APP_ROOT / "data" / "molecular_crystal" / "X23"

METRICS_CONFIG_PATH = Path(__file__).with_name("metrics.yml")
DEFAULT_THRESHOLDS, DEFAULT_TOOLTIPS, DEFAULT_WEIGHTS = load_metrics_config(
    METRICS_CONFIG_PATH
)

# Unit conversion
EV_TO_KJ_PER_MOL = units.mol / units.kJ

We define a function to retrieve system names, which is used to label points in the plot:

In [ ]:
def get_system_names() -> list[str]:
    """
    Get list of X23 system names.

    Returns
    -------
    list[str]
        List of system names from structure files.
    """
    system_names = []
    for model_name in MODELS:
        model_dir = CALC_PATH / model_name
        if model_dir.exists():
            xyz_files = sorted(model_dir.glob("*.xyz"))
            if xyz_files:
                for xyz_file in xyz_files:
                    atoms = read(xyz_file)
                    system_names.append(atoms.info["system"])
                break
    return system_names

We use pytest fixtures to build a dependency chain, building up from the raw data. The first level above raw data is plotting the predicted vs reference energies scatter plot.

We first define a function `lattice_energies()` to retrieve predicted and reference lattice energies from the calc outputs. The `@plot_parity` decorator tells the analysis framework to build a parity plot from the returned predicted and reference energies in a reproducible way. Here we add infomation such as the plot output path, title, axis labels and hover data (data shown when hovering over a point in the plot).

In [ ]:
@pytest.fixture
@plot_parity(
    filename=OUT_PATH / "figure_lattice_energies.json",
    title="X23 Lattice Energies",
    x_label="Predicted lattice energy / kJ/mol",
    y_label="Reference lattice energy / kJ/mol",
    hoverdata={
        "System": get_system_names(),
    },
)
def lattice_energies() -> dict[str, list]:
    """
    Get lattice energies for all X23 systems.

    Returns
    -------
    dict[str, list]
        Dictionary of reference and predicted lattice energies.
    """
    results = {"ref": []} | {mlip: [] for mlip in MODELS}
    ref_stored = False

    for model_name in MODELS:
        model_dir = CALC_PATH / model_name

        if not model_dir.exists():
            continue

        xyz_files = sorted(model_dir.glob("*.xyz"))
        if not xyz_files:
            continue

        for xyz_file in xyz_files:
            structs = read(xyz_file, index=":")

            solid_energy = structs[0].get_potential_energy()
            num_molecules = structs[0].info["num_molecules"]
            system = structs[0].info["system"]
            molecule_energy = structs[1].get_potential_energy()

            lattice_energy = (solid_energy / num_molecules) - molecule_energy
            results[model_name].append(lattice_energy * EV_TO_KJ_PER_MOL)

            # Copy individual structure files to app data directory
            structs_dir = OUT_PATH / model_name
            structs_dir.mkdir(parents=True, exist_ok=True)
            write(structs_dir / f"{system}.xyz", structs)

            # Store reference energies (only once)
            if not ref_stored:
                results["ref"].append(structs[0].info["ref"])

        ref_stored = True

    return results

Moving up the levels of abstraction, we define a function to compute the MAE metric from the predicted and reference lattice energies.

In [ ]:
@pytest.fixture
def x23_errors(lattice_energies) -> dict[str, float]:
    """
    Get mean absolute error for lattice energies.

    Parameters
    ----------
    lattice_energies
        Dictionary of reference and predicted lattice energies.

    Returns
    -------
    dict[str, float]
        Dictionary of predicted lattice energy errors for all models.
    """
    results = {}
    for model_name in MODELS:
        if lattice_energies[model_name]:
            results[model_name] = mae(
                lattice_energies["ref"], lattice_energies[model_name]
            )
        else:
            results[model_name] = None
    return results

Next, we build a results table from the MAEs using the `@build_table` decorator. For tests not using dispersion corrections, `mlip_name_map` can be omitted or set to `None`.

In [ ]:
@pytest.fixture
@build_table(
    filename=OUT_PATH / "x23_metrics_table.json",
    metric_tooltips=DEFAULT_TOOLTIPS,
    thresholds=DEFAULT_THRESHOLDS,
    mlip_name_map=D3_MODEL_NAMES,
)
def metrics(x23_errors: dict[str, float]) -> dict[str, dict]:
    """
    Get all X23 metrics.

    Parameters
    ----------
    x23_errors
        Mean absolute errors for all systems.

    Returns
    -------
    dict[str, dict]
        Metric names and values for all models.
    """
    return {
        "MAE": x23_errors,
    }

This final fucntion must begin with `test_` to be discovered by `pytest` and execute the analysis chain. You can see how the chain is formed from the function arguments, where each argument is a dependency on a previous fixture.

In [ ]:
def test_x23(metrics: dict[str, dict]) -> None:
    """
    Run X23 test.

    Parameters
    ----------
    metrics
        All X23 metrics.
    """
    return

You can now run the analysis through the CLI:

```bash
    ml_peg analyse --category molecular_crystal --test X23
```
See the [CLI documentation](https://ddmms.github.io/ml-peg/developer_guide/running.html) for more details.

## 3. App step: building the interactive app

Now that we have prepared and saved the table and plots for our new benchmark, the final step is to create an application, which defines the layout and interactivity for users to explore.

First create a new directory for your benchmark in `ml_peg/app/<category>/your_benchmark/` and create a new Python script `app_your_benchmark.py` in this directory.

As with calculations and analysis, the initial setup is relatively standard across benchmarks. The main things to change are:

- `ml_peg.app.utils.build_callbacks` imports
  - These will depend on the interactivity your benchmark requires
- `BENCHMARK_NAME`
  - This defines the human-friendly name for your benchmark
- `DOCS_URL`
  - This defines the link to benchmarks documentation
  - The final part of the URL must be `[catgeory].html#[benchmark-subheading]`, where the subheading corresponds to the benchmark's subheading in the category's documentation .rst file. This may differ to name used for your benchmark directory, and any spaces are replaced with `-`. For example, [Elemental Slab Oxygen Adsorption](https://ddmms.github.io/ml-peg/user_guide/benchmarks/surfaces.html#elemental-slab-oxygen-adsorption) would have `surfaces.html#elemental-slab-oxygen-adsorption`.
- `DATA_PATH`
  - This should correspond to where data for the application has been saved

In [ ]:
"""Run X23 app."""

from __future__ import annotations

from dash import Dash
from dash.html import Div

from ml_peg.app import APP_ROOT
from ml_peg.app.base_app import BaseApp
from ml_peg.app.utils.build_callbacks import (
    plot_from_table_column,
    struct_from_scatter,
)
from ml_peg.app.utils.load import read_plot
from ml_peg.models.get_models import get_model_names
from ml_peg.models.models import current_models

# Get all models
MODELS = get_model_names(current_models)
BENCHMARK_NAME = "X23 Lattice Energies"
DOCS_URL = (
    "https://ddmms.github.io/ml-peg/user_guide/benchmarks/molecular_crystal.html#x23"
)
DATA_PATH = APP_ROOT / "data" / "molecular_crystal" / "X23"

Next, we define the benchmark app's layout and interactivity by creating a new class that inherits from `BaseApp`. `BaseApp` automatically handles building the app layout, so in practice the only requirement should be to define the `register_callbacks` function, which is how we specify the app's interactivity.

In this case, we need to define the interactivity that enables:

1. Clicking on the table showing the scatter plot

For this, we use the `plot_from_table_column` function imported from `ml_peg.app.utils.build_callbacks`. This requires:

- An ID for the data table being clicked, which is automatically defined for the app as `self.table_id`
- A placeholder ID for the scatter plot. This should always include `BENCHMARK_NAME`, as IDs must be unique across all benchmark applications
- A mapping from the table column name(s) to the scatter plot(s) that should be viewed when the corresponding column is clicked. Each dictionary key in `column_to_plot` corresponds to a table column header, and the dictionary value corresponds to the scatter plot loaded using `read_plot`. Note that we call `read_plot` within `register_callbacks`, as if there are any issues loading this, a non-interactive form of the data table can still be viewed when using the command-line interface

2. Clicking on a point in the scatter plot showing the corresponding structure

For this, we use the `struct_from_scatter` function `struct_from_scatter`imported from `ml_peg.app.utils.build_callbacks`. This requires:

- The ID for the loaded scatter plot. This is the `id` passed to `read_plot`, not the placeholder ID, but similarly should include `BENCHMARK_NAME` to ensure it is unique
- A unique placeholder ID for the visualisation
- A list of structure files, in the same order as the scatter points where defined. These will be saved in `assets/[category]/[your benchmark]/[model name]`. Since the list of structures is the same for all models, in this case we simply find all of the structures saved for the first model, and sort this consistently with the sorting done within analysis. Note that `sorted` by default sorts lexicographically, not numerically, but this is not a problem if this is consistent with the order of the scatter points
- Whether each structure file contains a single structure (`struct`), or multiple structures to be visualised per scatter point (`traj`)

Further pre-defined callbacks are available in `ml_peg.app.utils.build_callbacks`, including cell-specific (rather than per column) interactivity.

In [ ]:
class X23App(BaseApp):
    """X23 benchmark app layout and callbacks."""

    def register_callbacks(self) -> None:
        """Register callbacks to app."""
        scatter = read_plot(
            DATA_PATH / "figure_lattice_energies.json",
            id=f"{BENCHMARK_NAME}-figure",
        )

        # Assets dir will be parent directory - individual files for each system
        structs_dir = DATA_PATH / MODELS[0]
        structs = [
            f"assets/molecular_crystal/X23/{MODELS[0]}/{struct_file.stem}.xyz"
            for struct_file in sorted(structs_dir.glob("*.xyz"))
        ]

        plot_from_table_column(
            table_id=self.table_id,
            plot_id=f"{BENCHMARK_NAME}-figure-placeholder",
            column_to_plot={"MAE": scatter},
        )

        struct_from_scatter(
            scatter_id=f"{BENCHMARK_NAME}-figure",
            struct_id=f"{BENCHMARK_NAME}-struct-placeholder",
            structs=structs,
            mode="struct",
        )

We then create a function that instantiates our new app class, which **must** be called `get_app`. This is where we pass the global variables defined at the start of our script, as well as a description of the benchmark, and the name of the saved data table to be loaded.

We also pass IDs of the placeholders for the additional interactive components we added above to `extra_components`, in the order that they should be displayed. In this case, these correspond to the placeholder for the scatter plot, followed by the placeholder for the structure visualisation.

In [ ]:
def get_app() -> X23App:
    """
    Get X23 benchmark app layout and callback registration.

    Returns
    -------
    X23App
        Benchmark layout and callback registration.
    """
    return X23App(
        name=BENCHMARK_NAME,
        description="Lattice energies for 23 organic molecular crystals.",
        docs_url=DOCS_URL,
        table_path=DATA_PATH / "x23_metrics_table.json",
        extra_components=[
            Div(id=f"{BENCHMARK_NAME}-figure-placeholder"),
            Div(id=f"{BENCHMARK_NAME}-struct-placeholder"),
        ],
    )

Finally, we provide a way to build and run the application, involving getting the layout of the app, and registering the callbacks that we defined above.

This section of code is **not** used when using the command-line interface to run the application, e.g.

```bash
    ml_peg app --category molecular_crystal
```

but is useful for testing your new bechmark's application.

While the default port for Dash applications is 8050, we recommend setting a different port here, as you cannot run multiple applications on the same port, and this process is not always stopped by the notebook.

In [ ]:
if __name__ == "__main__":
    # Create Dash app
    full_app = Dash(__name__, assets_folder=DATA_PATH.parent.parent)

    # Construct layout and register callbacks
    x23_app = get_app()
    full_app.layout = x23_app.layout
    x23_app.register_callbacks()

    # Run app
    full_app.run(port=8055, debug=True)

## Next steps

We encourage you to read through our online [developer documentation](https://ddmms.github.io/ml-peg/developer_guide/index.html) for further help, including:

- Setting up [developer tools](https://ddmms.github.io/ml-peg/developer_guide/get_started.html)
- [Adding benchmarks](https://ddmms.github.io/ml-peg/developer_guide/add_benchmarks.html)
  - This covers similar material to this tutorial, but provides more context on some underlying features such as the decorators used in analysis, and alternative calculation drivers
- Running ML-PEGs [calculations, analysis, and application](https://ddmms.github.io/ml-peg/developer_guide/running.html)
- [Adding data required for calculations](https://ddmms.github.io/ml-peg/developer_guide/data.html)
- Benchmark [scoring and normalisation](https://ddmms.github.io/ml-peg/developer_guide/scoring_and_normalisation.html)

You may also find it useful to refer to some [exemplar pull requests](https://github.com/ddmms/ml-peg/pulls?q=is%3Apr+label%3A%22example+benchmark+addition%22+), incuding examples of [data-only additions](https://github.com/ddmms/ml-peg/pulls?q=+is%3Apr+label%3A%22data+only%22+).